# Data Warehouse Medicare National QA - Claim Diag

## Initialization

Just loading packages that will be used and initializing connection to GP DB.

In [1]:
import pandas as pd
import sys
import psycopg2
from tqdm import tqdm
sys.path.append('H:/uth_helpers')
from db_utils import get_dsn

In [2]:
connection = psycopg2.connect(get_dsn())
connection.autocommit = True

## Row Count and Claim Count

In [3]:
query = ''' drop table if exists qa_reporting.dw_mcrn_claim_diag_counts;
create table qa_reporting.dw_mcrn_claim_diag_counts
(
    calendar_year int,
    dw_row_count bigint,
    src_row_count bigint,
    row_count_diff bigint,
    row_count_percentage float,
    dw_uth_clm_id_count bigint,
    dw_src_clm_id_count bigint,
    src_clm_count bigint,
    clm_count_diff bigint,
    clm_count_percentage float,
    dw_uth_mbr_id_count bigint,
    dw_src_mbr_id_count bigint,
    src_mbr_count bigint,
    mbr_count_diff bigint,
    mbr_count_percentage float,
    date_generated date
);
'''

with connection.cursor() as cursor:
    cursor.execute(query)

In [4]:
with connection.cursor() as cursor:
    query = f'''
    insert into qa_reporting.dw_mcrn_claim_diag_counts
    (calendar_year, dw_row_count, dw_uth_clm_id_count, date_generated)
    select year, count(*), count(distinct uth_claim_id), current_date
    from dw_staging.mcrn_claim_diag
    where diag_position = '1'
    group by 1
    '''
    
    cursor.execute(query)

    query = f'''
    update qa_reporting.dw_mcrn_claim_diag_counts b
    set dw_src_clm_id_count = count
    from (
        select year,  count(distinct claim_id_src) as count 
        from dw_staging.mcrn_claim_diag
    group by 1) a
    where a.year = b.calendar_year
    '''

    cursor.execute(query)

    query = f'''
    update qa_reporting.dw_mcrn_claim_diag_counts b
    set dw_uth_mbr_id_count = count
    from (
        select year, count(distinct uth_member_id) as count 
        from dw_staging.mcrn_claim_diag
    group by 1) a
    where a.year = b.calendar_year
    '''

    cursor.execute(query)
    
    query = f'''
    update qa_reporting.dw_mcrn_claim_diag_counts b
    set dw_src_mbr_id_count = count
    from (
        select year, count(distinct member_id_src) as count 
        from dw_staging.mcrn_claim_diag
    group by 1) a
    where a.year = b.calendar_year
    '''

    cursor.execute(query)

In [5]:
with connection.cursor() as cursor:
    query = '''
    with clms as (
        select extract(year from clm_thru_dt::date) as year, bene_id, clm_id
        from medicare_national.hha_base_claims_k
        where icd_dgns_cd1 is not null
        union
        select extract(year from clm_thru_dt::date) as year, bene_id, clm_id
        from medicare_national.outpatient_base_claims_k
        where icd_dgns_cd1 is not null
        union
        select extract(year from clm_thru_dt::date) as year, bene_id, clm_id
        from medicare_national.dme_claims_k
        where icd_dgns_cd1 is not null
        union
        select extract(year from clm_thru_dt::date)as year, bene_id, clm_id
        from medicare_national.inpatient_base_claims_k
        where icd_dgns_cd1 is not null
        union
        select extract(year from clm_thru_dt::date) as year, bene_id, clm_id
        from medicare_national.bcarrier_claims_k
        where icd_dgns_cd1 is not null
        union
        select extract(year from clm_thru_dt::date) as year, bene_id, clm_id
        from medicare_national.hospice_base_claims_k
        where icd_dgns_cd1 is not null
        union
        select extract(year from clm_thru_dt::date) as year, bene_id, clm_id
        from medicare_national.snf_base_claims_k
        where icd_dgns_cd1 is not null
    ),
    clm_counts as (
        select year, count(*) row_count, count(distinct bene_id) pat_count, count(distinct clm_id) clm_count
        from clms
        group by 1
    )
    update qa_reporting.dw_mcrn_claim_diag_counts a
    set src_row_count = b.row_count,
    row_count_diff = a.dw_row_count - b.row_count,
    row_count_percentage = 100. * abs(a.dw_row_count - b.row_count) / b.row_count,
    src_clm_count = b.clm_count,
    clm_count_diff = a.dw_uth_clm_id_count - b.clm_count,
    clm_count_percentage = 100. * abs(a.dw_uth_clm_id_count - b.clm_count) / b.clm_count,
    src_mbr_count = b.pat_count,
    mbr_count_diff = a.dw_uth_mbr_id_count - b.pat_count,
    mbr_count_percentage = 100. * abs(a.dw_uth_mbr_id_count - b.pat_count) / b.pat_count
    from clm_counts b
    where a.calendar_year = b.year
    ;
    '''

    cursor.execute(query)

In [6]:
query = '''select * from qa_reporting.dw_mcrn_claim_diag_counts;'''

df = pd.read_sql(query, con=connection)
df.sort_values(['clm_count_percentage'], ascending=False)

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,calendar_year,dw_row_count,src_row_count,row_count_diff,row_count_percentage,dw_uth_clm_id_count,dw_src_clm_id_count,src_clm_count,clm_count_diff,clm_count_percentage,dw_uth_mbr_id_count,dw_src_mbr_id_count,src_mbr_count,mbr_count_diff,mbr_count_percentage,date_generated
5,2021,58325504,58409170.0,-83666.0,0.143241,58325502,58325504,58409170.0,-83668.0,0.143245,2511213,2511215,2511831.0,-618.0,0.024604,2023-12-18
8,2014,58037679,58057405.0,-19726.0,0.033977,58037679,58037679,58057405.0,-19726.0,0.033977,1852503,1852503,1853140.0,-637.0,0.034374,2023-12-18
0,2018,60197620,60210464.0,-12844.0,0.021332,60197620,60197620,60210464.0,-12844.0,0.021332,1930824,1930824,1931573.0,-749.0,0.038777,2023-12-18
2,2020,53806637,53797856.0,8781.0,0.016322,53806637,53806637,53797856.0,8781.0,0.016322,1860139,1860139,1860824.0,-685.0,0.036812,2023-12-18
10,2019,60543418,60550050.0,-6632.0,0.010953,60543418,60543418,60550050.0,-6632.0,0.010953,1910455,1910455,1910942.0,-487.0,0.025485,2023-12-18
15,2017,59673909,59677566.0,-3657.0,0.006128,59673909,59673909,59677566.0,-3657.0,0.006128,1910059,1910059,1910438.0,-379.0,0.019838,2023-12-18
21,2015,58864100,58866192.0,-2092.0,0.003554,58864100,58864100,58866192.0,-2092.0,0.003554,1873335,1873335,1873851.0,-516.0,0.027537,2023-12-18
18,2016,59668121,59670191.0,-2070.0,0.003469,59668121,59668121,59670191.0,-2070.0,0.003469,1899201,1899201,1899637.0,-436.0,0.022952,2023-12-18
1,2000,2,NaN,NaN,NaN,2,2,NaN,NaN,NaN,2,2,NaN,NaN,NaN,2023-12-18
3,2002,4,NaN,NaN,NaN,4,4,NaN,NaN,NaN,4,4,NaN,NaN,NaN,2023-12-18


## Diagnosis Codes

Here we will check if we have valid ICD Diagnosis codes using our reference tables.

In [7]:
query = '''drop table if exists qa_reporting.dw_mcrn_diag_counts;
select year, diag_cd, count(*) as diag_count
into qa_reporting.dw_mcrn_diag_counts
from dw_staging.mcrn_claim_diag
group by 1,2;
'''

with connection.cursor() as cursor:
    cursor.execute(query)

In [8]:
diag_cd_df = pd.read_sql('select * from qa_reporting.dw_mcrn_diag_counts;', con=connection)
diag_cd_df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,diag_cd,diag_count
0,2020,S52042S,1
1,2015,E783,875
2,2022,K5792,4
3,2015,S140XXD,4
4,2021,P969,14
...,...,...,...
339946,2014,71121,2
339947,2010,2689,1
339948,2017,S62639G,2
339949,2015,64701,1


We see that we have a lot of claims with diagnosis codes that are not in our reference list. If we take a further look at some of these codes, we see that in our reference list, there is at least one more digit missing from these codes. The missing digits helps specify the diagnosis code.

In [9]:
query = '''
select a.*
from qa_reporting.dw_mcrn_diag_counts a
left join reference_tables.ref_cms_icd_cm_codes b
on a.diag_cd = cd_value
where b.cd_value is null;
'''

invalid_diag_df = pd.read_sql(query, con=connection)
invalid_diag_df

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,diag_cd,diag_count
0,2017,J49,1
1,2017,S283,2
2,2014,36430,2
3,2019,J49,1
4,2015,4273102,1
...,...,...,...
1518,2015,2422,1
1519,2014,7178,1
1520,2019,H522212,1
1521,2017,S32XXXA,1


Overall, the number of invalid diagnosis codes is neglible compared to the overall number of diagnosis codes in the claim_diag table.

In [10]:
invalid_diag_df.groupby('year')['diag_count'].sum()

year
2013       1
2014    1327
2015    1064
2016     178
2017     220
2018     165
2019     223
2020      84
2021     123
Name: diag_count, dtype: int64

In [11]:
diag_comp_df = pd.DataFrame({'overall_diag_count': diag_cd_df.groupby('year')['diag_count'].sum(),
                            'invalid_diag_count': invalid_diag_df.groupby('year')['diag_count'].sum(),
                            'valid_diag_count': diag_cd_df.groupby('year')['diag_count'].sum() - invalid_diag_df.groupby('year')['diag_count'].sum()})
diag_comp_df.loc[diag_comp_df['valid_diag_count'].isna(),'valid_diag_count'] = diag_comp_df.loc[diag_comp_df['valid_diag_count'].isna(),'overall_diag_count']
diag_comp_df['valid_diag_count'] =  diag_comp_df['valid_diag_count'].astype(int)
diag_comp_df['invalid_to_valid_percent'] = 100. * diag_comp_df['invalid_diag_count'] / diag_comp_df['valid_diag_count']
diag_comp_df  

,overall_diag_count,invalid_diag_count,valid_diag_count,invalid_to_valid_percent
year,,,,
1997,25,NaN,25,NaN
2000,18,NaN,18,NaN
2001,32,NaN,32,NaN
2002,70,NaN,70,NaN
2003,27,NaN,27,NaN
2004,66,NaN,66,NaN
2005,1,NaN,1,NaN
2006,63,NaN,63,NaN
2007,31,NaN,31,NaN


In [12]:
invalid_diag_df.groupby('year').max()

,diag_cd,diag_count
year,,
2013,250,1
2014,XX000,475
2015,Z877440,381
2016,i8511,10
2017,ZG894,18
2018,ZI211,17
2019,Z96561,20
2020,Z96561,7
2021,ZOO6,7


## Diagnosis Code Position

In [13]:
query = '''drop table if exists qa_reporting.dw_mcrn_diag_position;
select year, diag_position, count(distinct uth_claim_id) as claim_count
into qa_reporting.dw_mcrn_diag_position
from dw_staging.mcrn_claim_diag
group by 1,2;
'''
with connection.cursor() as cursor:
    cursor.execute(query)


In [14]:
diag_position_df = pd.read_sql('select * from qa_reporting.dw_mcrn_diag_position;', con=connection)
diag_position_df.sort_values(['year', 'diag_position'])

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,diag_position,claim_count
411,1997,1,3
309,1997,2,3
157,1997,3,3
131,1997,4,3
31,1997,5,3
...,...,...,...
279,2022,5,143
59,2022,6,133
28,2022,7,113
440,2022,8,104


In [15]:
diag_position_df['diag_position'].unique()

array(['17', '18', '6', '2', '11', '23', '13', '10', '22', '21', '5',
       '16', '9', '19', '14', '25', '12', '7', '4', '1', '15', '8', '3',
       '20', '24'], dtype=object)

Checking if the counts for the diag_position are correct. The higher the diag_position is, the less counts there should be. If we sort the counts of the diag_position by year and assign their order, this value should match with the diag_position value.

In [19]:
diag_position_df['row_rank'] = diag_position_df.sort_values(['year', 'claim_count'], ascending=[True, False]).groupby(['year']).cumcount()+1
diag_position_df['position_check'] = diag_position_df['row_rank'] == diag_position_df['diag_position'].astype(int)
diag_position_df[~diag_position_df['position_check']]

,year,diag_position,claim_count,row_rank,position_check
4,2009,11,3,10,False
5,2022,23,7,21,False
7,2001,11,2,4,False
10,2002,21,2,13,False
13,2006,5,5,1,False
...,...,...,...,...,...
475,2004,9,5,8,False
479,2004,2,5,9,False
482,2002,13,2,23,False
486,2003,2,3,9,False


In [20]:
diag_position_df[~diag_position_df['position_check']].sort_values('claim_count', ascending=False)

,year,diag_position,claim_count,row_rank,position_check
463,2010,8,26,9,False
208,2010,9,26,8,False
139,2010,12,13,11,False
399,2010,11,13,12,False
138,2022,19,11,20,False
...,...,...,...,...,...
62,2007,18,1,7,False
179,2007,16,1,11,False
369,2007,9,1,16,False
371,2007,8,1,17,False


In [21]:
diag_position_df[~diag_position_df['position_check']]['year'].unique()

array([2009, 2022, 2001, 2002, 2006, 2000, 2010, 2008, 1997, 2004, 2007,
       2003, 2011, 2012], dtype=int64)

## ICD Version

In [22]:
query = '''
drop table if exists qa_reporting.dw_mcrn_icd_version_count;

select year, icd_version, count(distinct uth_claim_id) as claim_count
into qa_reporting.dw_mcrn_icd_version_count
from dw_staging.mcrn_claim_diag
group by 1,2;
'''
with connection.cursor() as cursor:
    cursor.execute(query)

In [23]:
icd_version_df = pd.read_sql('select * from qa_reporting.dw_mcrn_icd_version_count;', con=connection)
icd_version_df.sort_values(['year', 'icd_version'])

c:\Users\iperez9\.conda\envs\ip_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,icd_version,claim_count
21,1997,None,3
31,2000,None,2
6,2001,9,1
14,2001,None,3
9,2002,None,4
15,2003,None,3
20,2004,9,4
41,2004,None,6
1,2005,9,1
34,2006,None,5
